## Работа с файлами конигурации

Документация на [ConfigParser](https://docs.python.org/3/library/configparser.html).

In [27]:
import configparser

In [28]:
config = configparser.ConfigParser()
config['DEFAULT'] = {'ServerAliveInterval': '45',
                     'Compression': 'yes',
                     'CompressionLevel': '9'}
config['bitbucket.org'] = {}
config['bitbucket.org']['User'] = 'hg'
config['topsecret.server.com'] = {}
topsecret = config['topsecret.server.com']
topsecret['Port'] = '50022'     
topsecret['ForwardX11'] = 'no'  
config['DEFAULT']['ForwardX11'] = 'yes'
with open('example.ini', 'w') as configfile:
    config.write(configfile)

In [29]:
%cat example.ini

[DEFAULT]
serveraliveinterval = 45
compression = yes
compressionlevel = 9
forwardx11 = yes

[bitbucket.org]
user = hg

[topsecret.server.com]
port = 50022
forwardx11 = no



In [35]:
config = configparser.ConfigParser()
config.read('example.ini')
print("Sections:", config.sections())
print("If 'bitbucket.org' section is here:", 'bitbucket.org' in config)
print("If 'bitbucket.com' section is here:", 'bitbucket.com' in config)
print("bitbucket.org - User value:", config['bitbucket.org']['User'])
print("DEFAULT - Compression value:", config['DEFAULT']['Compression'])
topsecret = config['topsecret.server.com']
print("topsecret.server.com - ForwardX11", topsecret['ForwardX11'])
print("bitbucket.org keys and values")
for key in config['bitbucket.org']:
    print(f"    {key:20}: {config['bitbucket.org'][key]}")


Sections: ['bitbucket.org', 'topsecret.server.com']
If 'bitbucket.org' section is here: True
If 'bitbucket.com' section is here: False
bitbucket.org - User value: hg
DEFAULT - Compression value: yes
topsecret.server.com - ForwardX11 no
bitbucket.org keys and values
    user                : hg
    serveraliveinterval : 45
    compression         : yes
    compressionlevel    : 9
    forwardx11          : yes


In [52]:
config = configparser.ConfigParser()

#config.add_section("DEFAULT")
config.set("DEFAULT", "serveraliveinterval", "55")
config.set("DEFAULT", "compression", "yes")
config.set("DEFAULT", "compressionlevel", "9")
config.set("DEFAULT", "forwardx11", "yes")
config.add_section("bitbucket.org")
config.set("bitbucket.org", "user", "hg")
config.add_section("topsecret.server.com")
config.set("topsecret.server.com", "port", "500221")
config.set("topsecret.server.com", "forwardx11", "no")

with open("example.ini", "w") as config_file:
    config.write(config_file)
    
print("getting values: ", config.get("DEFAULT", "compression"),
      config.get("topsecret.server.com", "port"))

getting values:  yes 500221


In [45]:
%cat example.ini

[DEFAULT]
serveraliveinterval = 55
compression = yes
compressionlevel = 9
forwardx11 = yes

[bitbucket.org]
user = hg

[topsecret.server.com]
port = 500221
forwardx11 = no



Все значения должны иметь строковый тип.

In [48]:
config.set("DEFAULT", "serveraliveinterval", 55)

TypeError: option values must be strings

Секция DEFAULT должна указываться явно.

In [50]:
config.set("serveraliveinterval", "55")


TypeError: option values must be strings

Секция DEFAULT должна указываться явно даже при чтении.

Вложенные секции разрешены.

In [51]:
config.get("serveraliveinterval", "55")


NoSectionError: No section: 'serveraliveinterval'

Секции и значения можно удалять.

В документации указано ещё много разных возможностей для работы.

In [53]:
config.remove_option("DEFAULT", "serveraliveinterval")

True

## Ведение журнала (логов) средствами Python

В библиотеке [logging](https://docs.python.org/3/library/logging.html) существует пять последовательных уровней логирования: DEBUG, INFO, WARNING, ERROR и CRITICAL. При создании журнала можно задать, начиная с какого уровля сообщения будут сохраняться в файл.

Как всегда - всё самое вкусное на [Хабре](https://habr.com/ru/post/513966/).

In [56]:
import logging

In [71]:
# add filemode="w" to overwrite
logging.basicConfig(filename="sample.log", filemode="w", level=logging.INFO)
 
logging.debug("This is a debug message")
logging.info("Informational message")
logging.error("An error has happened!")

In [76]:
logger_main = logging.getLogger(__name__)
logger_main.error("Something 1")

logger_ex = logging.getLogger("ex")
logger_ex.setLevel(logging.WARNING)
logger_ex.error("Something 2")

In [77]:
log_format = f"%(asctime)s - [%(levelname)s] - %(name)s - (%(filename)s).%(funcName)s(%(lineno)d) - %(message)s"

logging.root.handlers[0].setFormatter(logging.Formatter(log_format))

In [85]:
logger_main.info("Test format 1")
logger_ex.error("Test format 2")

In [73]:
logging.root.handlers[0].close()

## Тестирование программ на Питоне

У Питона есть две встроенные возможности для проверки корректности выполнения программы. Первая из них - это оператор assert. Он проверяет истинность переданного выражения, и если оно ложно, останавливает выполнение программы.

In [1]:
assert 2==3, "That is the question"

AssertionError: That is the question

Однако такие проверки скорее походят на проверку корректного состояния программы, чем на тестирование. Тесты для тестирования пишутся и выполняются отдельно. В соответствии с методологией Agile, сперва пишутся тесты, а только потом код, который проверяется этими тестами.

Простейшим видом тестов являются модульные тесты, которые проверяют работу отдельных функций и модулей. Более сложными являются интеграционные тесты, проверяющие корректность взаимодействия модулей между собой. Здесь мы разберем инструментарий для модульного тестирования, однако в него могут быть внесены и интеграционные тесты.

#### Библиотека unittest

[Документация на unittest](https://docs.python.org/3/library/unittest.html).

Альтернатива - [doctest](https://docs.python.org/3/library/doctest.html#module-doctest), для которого тесты пишутся в строках документации к функциям. Но всё, что он может, это тестировать функции.

In [55]:
import unittest
# Just for fun.
from abc import abstractmethod
# Need for an example.
import sys

In [3]:
class TestNotebook(unittest.TestCase):
    
    def test_add(self):
        self.assertEqual(2+2, 4)
        
    def test_mul(self):
        self.assertEqual(2*2, 4)


In [4]:
unittest.main()

E
ERROR: /home/edward/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/home/edward/'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

/home/edward/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok
test_mul (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [6]:
class TestNotebook2(unittest.TestCase):
    
    def add2(self):
        self.assertEqual(2+2, 4)
        
    def mul2(self):
        self.assertEqual(2*2, 6)


In [7]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok
test_mul (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


In [8]:
class TestNotebook2(unittest.TestCase):
    
    def anyOther(self):
        some_code = 42
    
    def testAdd2(self):
        self.assertEqual(2+2, 4)
        
    def testMul2(self):
        self.assertEqual(2*2, 6)


In [9]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok
test_mul (__main__.TestNotebook) ... ok
testAdd2 (__main__.TestNotebook2) ... ok
testMul2 (__main__.TestNotebook2) ... FAIL

FAIL: testMul2 (__main__.TestNotebook2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-7a234d38096b>", line 10, in testMul2
    self.assertEqual(2*2, 6)
AssertionError: 4 != 6

----------------------------------------------------------------------
Ran 4 tests in 0.010s

FAILED (failures=1)


In [10]:
class TestNotebook3(unittest.TestCase):
    
    def testadd3(self):
        self.assertTrue(2+2==4)
        
    def testmul3(self):
        self.assertFalse(2*2==6)


In [11]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_add (__main__.TestNotebook) ... ok
test_mul (__main__.TestNotebook) ... ok
testAdd2 (__main__.TestNotebook2) ... ok
testMul2 (__main__.TestNotebook2) ... FAIL
testadd3 (__main__.TestNotebook3) ... ok
testmul3 (__main__.TestNotebook3) ... ok

FAIL: testMul2 (__main__.TestNotebook2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-7a234d38096b>", line 10, in testMul2
    self.assertEqual(2*2, 6)
AssertionError: 4 != 6

----------------------------------------------------------------------
Ran 6 tests in 0.009s

FAILED (failures=1)



|Method|Checks that|
|:-----------|:-------------|
|assertEqual(a, b)|a == b|
|assertNotEqual(a, b)|a != b|
|assertTrue(x)|bool(x) is True|
|assertFalse(x)|bool(x) is False|
|assertIs(a, b)|a is b|
|assertIsNot(a, b)|a is not b|
|assertIsNone(x)|x is None|
|assertIsNotNone(x)|x is not None|
|assertIn(a, b)|a in b|
|assertNotIn(a, b)|a not in b|
|assertIsInstance(a, b)|isinstance(a, b)|
|assertNotIsInstance(a, b)|not isinstance(a, b)|
|assertRaises(exc, fun, \*args, \*\*kwds)|fun(\*args, \*\*kwds) raises exc|
|assertRaisesRegex(exc, r, fun, \*args, \*\*kwds)|fun(\*args, \*\*kwds) raises exc and the message matches regex r|
|assertWarns(warn, fun, \*args, \*\*kwds)|fun(\*args, \*\*kwds) raises warn|
|assertWarnsRegex(warn, r, fun, \*args, \*\*kwds)|fun(\*args, \*\*kwds) raises warn and the message matches regex r|
|assertLogs(logger, level)|The with block logs on logger with minimum level|

In [12]:
class Clause:
    stringRepr = ''
    globalId = 0
    
    def __init__(self, _srepr):
        self.stringRepr = _srepr
        self.id = Clause.globalId + 1
        Clause.globalId += 1
    
    @abstractmethod
    def getStringRepr(self):
        pass
    

class WordClause(Clause):
    def getStringRepr(self):
        return self.stringRepr
    
class ComplexClause(Clause):
    def __init__(self, _srepr):
        Clause.__init__(self, _srepr)
        self.childs = []
    
    def getStringRepr(self):
        tstr = self.stringRepr + '['
        for c in self.childs:
            tstr += c.getStringRepr() + ','
        if len(self.childs) != 0:
            tstr = tstr[:-1]
        tstr += ']'
        return tstr
    
    def addClause(self, cla):
        self.childs.append(cla)

In [13]:
class ClauseTest(unittest.TestCase):
    def setUp(self):
        print("setting up an object")
        self.cl1=WordClause('123')
        self.cl2=WordClause('234')
        self.cl3=WordClause('345')
        self.cl4=WordClause('456')
        self.ccl1=ComplexClause('qwe')
        self.ccl2=ComplexClause('asd')
        self.ccl3=ComplexClause('zxc')

    @classmethod
    def setUpClass(cls):
        print("setting up the class")
    
    def tearDown(self):
        print("tearing down")
        
    def doCleanups(self):
        '''
    If setUp() fails, meaning that tearDown() is not called, then any cleanup functions added will still be called.
    addCleanup(function, /, *args, **kwargs)

    Add a function to be called after tearDown() to cleanup resources used during the test. Functions will be called in reverse order to the order they are added (LIFO). They are called with any arguments and keyword arguments passed into addCleanup() when they are added.
        '''
        print("cleaning up")

#     def shortDescription(self):
#         return "This is a testcase for word clauses."

    def testComplesClauseAddTest(self):
        '''test name in docstring'''
        print("start testing process 1")
        self.ccl1.addClause(self.cl1)
        self.assertEqual(len(self.ccl1.childs), 1)
        self.ccl1.addClause(self.cl2)
        self.assertEqual(len(self.ccl1.childs), 2)
        self.ccl2.addClause(self.cl3)
        self.ccl2.addClause(self.cl4)
        self.ccl3.addClause(self.ccl1)
        self.assertEqual(len(self.ccl3.childs), 1)
        self.ccl3.addClause(self.ccl2)
        self.assertEqual(len(self.ccl3.childs), 2)
        print("finish testing process 1")
        
    def testWordClauseStringReprTest(self):
        print("start testing process 2")
        self.assertEqual(self.cl1.getStringRepr(), '123')
        self.assertEqual(self.cl2.getStringRepr(), '234')
        self.assertEqual(self.ccl3.getStringRepr(), 'zxc[qwe[123,234],asd[345,456]]')
        print("finish testing process 2")
        # with self.subTest(i=i): - позволит проолжить тест.

    def testWordClauseStringReprTest2(self):
        print("start testing process 3")
        self.ccl1.addClause(self.cl1)
        self.ccl1.addClause(self.cl2)
        self.ccl2.addClause(self.cl3)
        self.ccl2.addClause(self.cl4)
        self.ccl3.addClause(self.ccl1)
        self.ccl3.addClause(self.ccl2)
        self.assertEqual(self.ccl3.getStringRepr(), 'zxc[qwe[123,234],asd[345,456]]')
        print("finish testing process 3")


In [14]:
unittest.main(argv=[''], verbosity=2, exit=False)

testComplesClauseAddTest (__main__.ClauseTest)
test name in docstring ... ok
testWordClauseStringReprTest (__main__.ClauseTest) ... FAIL
testWordClauseStringReprTest2 (__main__.ClauseTest) ... ok
test_add (__main__.TestNotebook) ... ok
test_mul (__main__.TestNotebook) ... ok
testAdd2 (__main__.TestNotebook2) ... ok
testMul2 (__main__.TestNotebook2) ... FAIL
testadd3 (__main__.TestNotebook3) ... ok
testmul3 (__main__.TestNotebook3) ... 

setting up the class
setting up an object
start testing process 1
finish testing process 1
tearing down
cleaning up
setting up an object
start testing process 2
tearing down
cleaning up
setting up an object
start testing process 3
finish testing process 3
tearing down
cleaning up


ok

FAIL: testWordClauseStringReprTest (__main__.ClauseTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-13-079424716a3f>", line 50, in testWordClauseStringReprTest
    self.assertEqual(self.ccl3.getStringRepr(), 'zxc[qwe[123,234],asd[345,456]]')
AssertionError: 'zxc[]' != 'zxc[qwe[123,234],asd[345,456]]'
- zxc[]
+ zxc[qwe[123,234],asd[345,456]]


FAIL: testMul2 (__main__.TestNotebook2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-7a234d38096b>", line 10, in testMul2
    self.assertEqual(2*2, 6)
AssertionError: 4 != 6

----------------------------------------------------------------------
Ran 9 tests in 0.017s

FAILED (failures=2)


In [25]:
def external_resource_available():
    return False

class TestSkipTest(unittest.TestCase):
    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(not sys.platform.startswith("win"), "requires Windows")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        if not external_resource_available():
            self.skipTest("external resource not available")
        # test code that depends on the external resource
        pass
    
    

# @unittest.expectedFailure
# Mark the test as an expected failure or error. 
# If the test fails or errors it will be considered a success. 
# If the test passes, it will be considered a failure.


In [26]:
unittest.main(argv=[''], verbosity=1, exit=False)

.F....F..ssss

setting up the class
setting up an object
start testing process 1
finish testing process 1
tearing down
cleaning up
setting up an object
start testing process 2
tearing down
cleaning up
setting up an object
start testing process 3
finish testing process 3
tearing down
cleaning up



FAIL: testWordClauseStringReprTest (__main__.ClauseTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-13-079424716a3f>", line 50, in testWordClauseStringReprTest
    self.assertEqual(self.ccl3.getStringRepr(), 'zxc[qwe[123,234],asd[345,456]]')
AssertionError: 'zxc[]' != 'zxc[qwe[123,234],asd[345,456]]'
- zxc[]
+ zxc[qwe[123,234],asd[345,456]]


FAIL: testMul2 (__main__.TestNotebook2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-8-7a234d38096b>", line 10, in testMul2
    self.assertEqual(2*2, 6)
AssertionError: 4 != 6

----------------------------------------------------------------------
Ran 13 tests in 0.012s

FAILED (failures=2, skipped=4)


> Если вы обнаружили, что в блоке кода, который вы хотите протестировать, много побочных эффектов, значит вы нарушаете <a href="https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF_%D0%B5%D0%B4%D0%B8%D0%BD%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B9_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D0%B8">Принцип Единственной Ответственности</a>. Нарушение принципа единственной ответственности означает, что фрагмент кода делает слишком много вещей и требует рефакторинга. Следование принципу единственной ответственности — отличный способ проектирования кода, для которого не составит труда писать простые повторяемые модульные тесты, и, в конечном счете, создания надежных приложений.  

<div align="right"><a href="https://habr.com/ru/company/otus/blog/433358/">https://habr.com/ru/company/otus/blog/433358/</a></div>

Принцип единой ответственности относится к [принципам](https://habr.com/ru/post/446816/) [SOLID](https://habr.com/ru/company/ruvds/blog/426413/) ([и вот ещё](https://habr.com/ru/company/mailru/blog/412699/)), помогающим писать хороший код на основе объектно-ориентированной парадигмы.

![](img/lbzrqyibpifgxpgagwl44tgw7gu.png)

